In [1]:
from pysat.formula import CNF
from pysat.solvers import MinisatGH

In [12]:
from itertools import combinations
'''
We make use of a class that we give facts (the setup of the board) to encode the problem in SAT.
Instead of A-E we make use of the numbers 1-5
We number each cell 1-25 starting from the top left corner and going row by row. 
Then for each square we have 5 possibilities. Cell 1 having value 1 will be encoded by variable 1.
Cell 1 having value 2 will be encoded by variable 2. Then cell 2 having value 1 will be encoded by 6 etc..

'''
class LatinSquare():
    def __init__(self, facts):
        self.formula = CNF()
        self.board = self.create_board()
        #First we encode that each cell must have exactly 1 number between 1 and 5 (A-E)
        self.encode_exact_1_number()
        
        # Each row contains distinct values
        self.row_unique()
        # Each column contains distinct values
        self.column_unique()
        # Add the facts to the model 
        self.add_facts(facts)

        
    def create_board(self):
        '''
        creates the board, used to quickly translate from the variable encoding to tupple encoding
        E.g. key: (3,4) has value: 14
        '''
        board = {}
        for cell in range(25):
            possible_values_cell = [cell * 5 + i for i in range(1,6)]

            for i in range(1,6):
                board[cell * 5 + i ] = (cell, i) # tuple row, columns
        return board
        
        
    def encode_exact_1_number(self):
        # Each cell must have one unique value. 
        for cell in range(25):
            possible_values_cell = [cell * 5 + i for i in range(1,6)]
            self.formula.append(possible_values_cell)
            #Next encode that each cell must have at most 1 possible value.
            self.add_exclusion_rule([possible_values_cell])
            
    def row_unique(self):
        # Each row must have distinct values. This get encoded by saying that 1 until 5 must appear exactly once in each row.
        for i in range(1,6):
            rows = self.get_row_one_value(i)
            self.add_exclusion_rule(rows)
            
    def column_unique(self):
        # Similar to row_unique. 
        for i in range(1,6):
            columns = self.get_column_one_value(i)
            self.add_exclusion_rule(columns)
            
    def add_facts(self, facts):
        board_inverse = {value: key for key, value in self.board.items()}
        
        for fact in facts:
            self.formula.append([board_inverse[fact]])
            
          
    def add_exclusion_rule(self, lists):
        '''
        Given a list of lists, will add a pairwise exclusion rule to the formula.
        
        E.g:
        [[1,2, 3], [5,6]] will add the rules:
        [-1, -2], [-1, -3], [-2, -3] and [-5,-6] to the formula
        This ensures that not both values can be true at the same time. 
        '''
        for l in lists:
            # Pairswise exclusion
            for combination in combinations(l, 2):
                p1 = combination[0]
                p2 = combination[1]
                 # not both can be true at the same time so at least one must be false.
                both_not_true = [-p1, -p2]
                self.formula.append(both_not_true)
                
                
                
    def get_row_one_value(self, value):
        rows_value = []
        possible_values_row = []
        for cell in range(25):
            possible_values_row.append(cell * 5 + value)
            if (cell + 1) % 5 == 0:
                rows_value.append(possible_values_row)
                
                possible_values_row = []
        return rows_value
    
    def get_column_one_value(self, value):
        columns_value = [ [] for i in range(5)]
        possible_values_row = []
        for cell in range(25):
            index = cell % 5 
            columns_value[index].append(cell * 5 + value)
        return columns_value

        
    def get_solutions(self):
        solver = MinisatGH()
        solver.append_formula(self.formula)
        for i, model in enumerate(solver.enum_models(), 1):
            print("MODEL #{}:".format(i))
            filled_in_values = []
            for lit in model:
                if lit > 0:
                    filled_in_values.append(lit)
            values = [self.board[value] for value in filled_in_values]
           
            self.pretty_print(values)
    def pretty_print(self, values):
        string = ""
        for expected_cell, (cell,value) in zip([i for i in range(25)], values):
            assert expected_cell == cell, "expected_cell != actual cell, {}, {}".format(expected_cell, cell)
            string += "|" + str(value) + "|"
            if (cell + 1) % 5 == 0:
                string += "\n"
        print(string)
 

         

In [13]:
def pretty_print(facts):
    string = ""
    index = 0
    (cell, value) = facts[index]
    for expected_cell in range(25):
        if expected_cell == cell:
            string += "|" + str(value) + "|"
            index += 1
            if index < len(facts):
                (cell, value) = facts[index]
        else:
            string += "|?|"
        if (expected_cell + 1) % 5 == 0:
            string += "\n"
    print(string)

In [14]:
# This is the board that we are given:
facts = [
    (0,1),
    (1, 3),
    (7,5),
    (9,1),
    (12,2),
    (15, 4),
    (17, 3),
    (23, 5),
    (24, 3),
    
]
pretty_print(facts)



|1||3||?||?||?|
|?||?||5||?||1|
|?||?||2||?||?|
|4||?||3||?||?|
|?||?||?||5||3|



In [15]:
# Initialize with the given facts
latin_square = LatinSquare(facts)

In [16]:
# get the solutions (which in this case is unique)
latin_square.get_solutions()

MODEL #1:
|1||3||4||2||5|
|3||2||5||4||1|
|5||1||2||3||4|
|4||5||3||1||2|
|2||4||1||5||3|

